> 05 블럭깨기
- 설명이 필요없는 유명한 게임입니다. 화면 아래에있는 패들을 조작해서 공을 튕겨서 화면위에 있는 블록을 전부 제거하면 됩니다

In [17]:
#blocks  second_version

import sys
import math
import random
import pygame
from pygame.locals import QUIT, KEYDOWN, K_LEFT, K_RIGHT, Rect

def start_screen(): ####시작준비화면####################
    from pygame.locals import QUIT, KEYDOWN, K_SPACE
    pygame.init()
    SURFACE = pygame.display.set_mode((600, 800))
    FPSCLOCK = pygame.time.Clock()
    
    pygame.display.set_caption("python pygame")
    # 글꼴 설정
    font = pygame.font.SysFont(None, 36)
    text = font.render("Start press SPACE!", True, (100,100,100))
    text_rect = text.get_rect(center=(600//2,800//2))

    # --- 1단계: 대기 화면 ---
    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                raise SystemExit
            if event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                waiting = False  # 스페이스키 눌리면 대기 종료
        SURFACE.fill((0,0,0))
        SURFACE.blit(text, text_rect)
        pygame.display.flip()
        FPSCLOCK.tick(30)
###################################################################################

pygame.mixer.init() 
hit_sound = pygame.mixer.Sound(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\1_sounds\bang.wav")   # 짧은 wav/mp3 파일
eat_sound = pygame.mixer.Sound(r"C:\KCS\Desktop\게임으로배우는 파이썬\Game\1_sounds\eat.wav")   # 짧은 wav/mp3 파일
# 실행 hit_sound.play()
#배경음악
# pygame.mixer.init()       # 음악 로드
# pygame.mixer.music.load("bgm.mp3")   # wav, mp3 둘 다 가능
# pygame.mixer.music.play(-1) # 반복 재생 ( -1 : 무한 반복 )
# pygame.mixer.music.set_volume(0.5) # 볼륨 조절 (0.0 ~ 1.0)
# #음악중지
# pygame.mixer.music.stop()

start_screen()  #시작준비화면 호출


class Block:  #블록, 공, 패들 객체을 한번에 정의가 가능하다고!  단색으로 빈틈없이 칠하는 일정 영역이 있다는 공통점이 있다.
    def __init__(self,col,rect,speed=0): #프로퍼티로  채우는색, 그리는 직사각형의 위치와 크기
        self.col= col
        self.rect= rect
        self.speed= speed #공일때만 속도가있다.
        self.dir= random.randint(-45,45)+270    #공일때만 방향이있다. 270도좌우로45도 즉,아래방향으로 랜덤
    def move(self): #공을움직인다.
        self.rect.centerx += math.cos(math.radians(self.dir)) * self.speed
        self.rect.centery -= math.sin(math.radians(self.dir)) * self.speed
    def draw(self): #블록, 공, 패들을 그린다.  #공과 블록, 패들은 그리는 방법이 다르다.속도가있으면 공이다.
        if self.speed==0: #블록과 패들
            pygame.draw.rect(SURFACE,self.col,self.rect)
        else: #공
            pygame.draw.ellipse(SURFACE,self.col,self.rect)

def tick(): #프레임별처리
    global BLOCKS
    for event in pygame.event.get():
        if event.type== QUIT:
            pygame.quit()
            sys.exit()
        elif event.type== KEYDOWN:
            if event.key== K_LEFT:
                PADDLE.rect.left-= 5
            elif event.key== K_RIGHT:
                PADDLE.rect.left+= 5
        
        #패들이 화면을 벗어나지 않토록 설정
        if PADDLE.rect.right < 10:
            PADDLE.rect.right= 10
        if PADDLE.rect.left > 590:
            PADDLE.rect.left= 590
            
    if BALL.rect.centery < 1000: #공이 화면안에 있을때만
        BALL.move()

    #블록과충돌
    prevlen = len(BLOCKS)   #충돌전의 블록 수
    BLOCKS =[x for x in BLOCKS if not x.rect.colliderect(BALL.rect)]
    if len(BLOCKS) != prevlen:
        eat_sound.play()
        BALL.dir *= -1

    #패들과충돌
    if PADDLE.rect.colliderect(BALL.rect):
        eat_sound.play()
        BALL.dir = 90 + (PADDLE.rect.centerx - BALL.rect.centerx) / PADDLE.rect.width * 80

    #벽과충돌
    if BALL.rect.centerx < 0 or BALL.rect.centerx > 600:
        BALL.dir = 180 - BALL.dir
    if BALL.rect.top < 0:
        BALL.dir = -BALL.dir
        BALL.speed = 15

pygame.init()
pygame.key.set_repeat(5,5)
SURFACE = pygame.display.set_mode((600, 800))
FPSCLOCK = pygame.time.Clock()
BLOCKS = [] #블록객체를 저장하는 리스트
PADDLE = Block((242,242,0),Rect(300,700,200,30))    #패들객체(블락클래스의인스턴스)
BALL = Block((242,242,100),Rect(300,400,20,20),10)

def main():
    myfont= pygame.font.SysFont(None,72)
    mess_clear = myfont.render("CLEARED!",True,(255,255,255))
    mess_over = myfont.render("GAME OVER",True,(255,255,255))
    fps = 30
    colors = [(255,0,0),(0,255,0),(0,0,255),(255,255,0),(255,0,255),(0,255,255)] #색깔을초기화

    for ypos,color in enumerate(colors,start=0): #(0,(255,0,0)),(1,(0,255,0))...    
        for xpos in range(0,5):
            BLOCKS.append(Block(color,Rect(xpos*100+60,ypos*50+40,80,30)))
    while True:
        tick()
        SURFACE.fill((0,0,0))
        BALL.draw()
        PADDLE.draw()
        for block in BLOCKS:
            block.draw()
        if len(BLOCKS)==0:
            SURFACE.blit(mess_clear,(200,350))
        if BALL.rect.top > 800 and len(BLOCKS)>0:
            hit_sound.play()
            SURFACE.blit(mess_over,(180,350))
        
        pygame.display.update()
        FPSCLOCK.tick(fps)

if __name__ == '__main__':
    main()

SystemExit: 